<a href="https://colab.research.google.com/github/sinnis1991/pose-estimation-colab-/blob/master/A_generative_feature_to_image_robotic_vision_framework_for_6_D_pose_measurement_of_metal_parts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><font size=6> A generative feature-to-image robotic vision framework for 6-D pose measurement of metal parts</font></center>  

---

<p align="left"> This notebook gives an example of the proposed 6-D pose estimation method. 
It is based a unique <font color=#04a1e5> generative feature-to-image framework</font> constructed through deep generative models. 
A specific algorithm is used to indirectly infer the pose through an <font color=#F08519>interactive regression pose estimation process </font>.
The code is divided into these 2 parts. We will give a brief instruction of these 2 parts of codes' work. 

<font size=2>(P.S. If you can't see the instruction clearly, please change the notebook style: Tools ==> Settings ==> Site ==> Theme ==> dark)</font>
<br/><br/>


<center><font size=5 color=#04a1e5> generative feature-to-image framework</font></center>  
<center><font color=#04a1e5>==================================================================================================================</font></center>
<img align="right" width="539" height="300" src="https://raw.githubusercontent.com/sinnis1991/colab-pose-VAE/master/ims/figure/1.png"/></img>

This framework represents a model (generator)： 

<center>${x_t} = G(z)$</center>  

whose input is a low-dimensional feature $z$ representing any pose $y$ freely, 
and the corresponding output is a standard generated image ${x_t}$ describing the object in pose $y$. 
In probability learning, it means to construct a conditioned probability model: 

<center>$x$~$p(x|z)$</center>

establishing a reverse mapping from feature to image,
as generative models. In particular, there's a restrction for the input $z$ :

<center>$z$~$q(z)$</center>

<br/><br/>

In this program, the model is specified as :

**Input :** feature <font color=#F08519>z</font>  (a 6-dimensional vector)  &nbsp; &nbsp; &nbsp;-----------> 
      &nbsp; &nbsp; &nbsp;**Output :** image <font color=#04a1e5>x</font>  (the corresponding binary contour image (128x128))

In this model, the input could represent any pose within the range and the generator could 
precisly generate the image of that pose, even that pose and image do not exist in its training dataset. <br/>

<font size = 6>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</font> 
<left><font size = 6>feature</font> <font size=6 color=#F08519>&nbsp;$z$</font>   (5th and 6th dimension) 
<font size = 6>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</font> 
<font size = 6>image</font> <font size=6 color=#04a1e5>&nbsp;$x$</font></left>


![Alt Text](https://raw.githubusercontent.com/sinnis1991/colab-pose-VAE/master/ims/figure/combine(32)_small.gif)


<br/><br/>

<center><font size=5 color=#F08519> interactive regression pose estimation process</font></center>  
<center><font color=#F08519>==================================================================================================================</font></center>


<img align="right" width="600" height="403" src="https://raw.githubusercontent.com/sinnis1991/colab-pose-VAE/master/ims/figure/2_move.gif"/></img>

This pose estimation process is based on the previously constructed generator model.<br/>
(1) It starts with a random feature and its generated image:<br/>
<center>( <font color=#F08519>${z_0}$</font>  , <font color=#04a1e5>${x_0}$</font> )</center>

(2) In the following iterations, it will interactively correct the input feature and the output generated image, to gradually regress them to the target answer.<br/>
<left>$z$ --> $x$ : from feature to generate image<br/>
$x$ <-- $z$ : backwardly correct the feature by estimating the distance with the target image</left>

(3) Finally, it will get an optimized feature and image:<br/>
<center>( <font color=#F08519>${z_n}$</font>  , <font color=#04a1e5>${x_n}$</font> )</center>

They are the final answers for the target pose.


<br/><br/>
In particualr, this estimation process can be **parallel**: 
<br/>initiate start points from multiple random z and converge to the same target. 
This gains a remarkble ensurance to get a right answer.


<font size = 6>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</font> 
<left><font size = 6>feature</font> <font size=6 color=#F08519>&nbsp;$z$</font>   (multiple) 
<font size = 6>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</font> 
<font size = 6>image</font> <font size=6 color=#04a1e5>&nbsp;$x$</font></left>


![Alt Text](https://raw.githubusercontent.com/sinnis1991/colab-pose-VAE/master/ims/figure/combine(complete)_small.gif)

In [ ]:
# Please mount your google drive. There must be at least 5GB free space.

from google.colab import drive
drive.mount('/content/gdrive')

<center><font size=6> Choose a target object for the task </font></center>
</br>

![ims](https://raw.githubusercontent.com/sinnis1991/colab-pose-VAE/master/ims/figure/models.png)




In [ ]:
# choose the index of the object

object_index = "1" 

In [ ]:
# import libararies

try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

!rm -rf colab-pose-VAE

import tensorflow as tf
from tensorflow import keras
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import json
import moviepy.editor as mvp
from google.colab import files
from lucid.misc.gl.glcontext import create_opengl_context

!git clone https://github.com/sinnis1991/colab-pose-VAE.git
import sys
sys.path.insert(0, "./colab-pose-VAE")
import os

from math import pi,cos,sin
from OpenGL.GL import *
from OpenGL.GLU import *
from opengl_model import estimate_3D_to_2D, gl_ob

from PIL import Image
from matplotlib import pyplot as plt